# Task 3: Evaluating the Model

Choose an evaluation metric fitting to your model and explain why you have chosen this and share your scores for this metric.

---

In this assessment we have considered AUC (Area Under the ROC Curve) since we are interested in ordering properly the output probabilities. 

In [3]:
import pickle
import pandas as pd

# load model from file
xgb = pickle.load(open("xgb.pickle.dat", "rb"))

In [5]:
df = pd.read_csv('./data/Courier_data_model.csv')

In [8]:
test = df[df['train']==0]

In [11]:
predictors = [x for x in df.columns if x not in ['courier', 'flag', 'feature_5', 'lifetime_1', 'train']]

In [16]:
test[predictors].values

array([[-0.57388295, -0.12869612,  0.57293063, ...,  0.14553626,
        -1.26587396,  0.50013976],
       [ 0.75842398,  0.66543146, -0.68932216, ...,  0.22771793,
        -0.02718161,  0.41027352],
       [-1.01798526,  0.84869167, -1.74119949, ..., -0.51191708,
         1.74417368, -0.26420207],
       ...,
       [ 1.20252628,  0.78760494,  0.65708082, ...,  0.22771793,
        -0.37844646,  0.42282144],
       [-1.46208757, -1.2282574 , -0.35272142, ...,  0.14553626,
        -0.25891471,  0.46416905],
       [-0.12978064,  0.48217125,  0.40463026, ...,  0.22771793,
        -0.44869943,  0.42533103]])

In [21]:
y_test_pred = xgb.predict_proba(test[predictors])[:,1]

In [22]:
test['pred'] = y_test_pred

/Users/moyandreu/coding/glovo/Data_Scientist_Interview/env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
y_test_pred = [round(value) for value in y_test_pred]

In [28]:
from sklearn.metrics import accuracy_score, roc_auc_score

# evaluate predictions
accuracy = accuracy_score(test['flag'].values, y_test_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("AUC: %.4f%%" % (100*roc_auc_score(test['flag'].values, y_test_pred)))

Accuracy: 77.89%
AUC: 76.6578%
